<a href="https://colab.research.google.com/github/dr-mushtaq/Chatbot-in-e-learning-system/blob/main/Model_Training_Mushtaq_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**Table of Content**</p>



1.   Import Library
2.   Import Dataset
3.   Data Preprocessing
4.   Model Training




#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**1-Import library**</p>

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**2-Import Dataset**</p>

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/gdrive/MyDrive/Research /Research Projects/Chatbot System in Education /Dataset/intents.json').read()
intents = json.loads(data_file)


#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**3-Data Preprocessing**</p>

In [8]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [9]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [11]:
 import nltk
 nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [12]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)



235 documents
93 classes [' ADMISSION OF FOREIGN/DUAL NATIONAL', ' Additional Specialization Certificate', ' Admission Status', ' Alumni Association ', ' Citizen complaint portal ', ' Club and Societies ', ' Courses Catalogue', ' Degree/Transcript Verification ', ' Digiskills ', ' Disabled Student Fee structure(Local)', ' Disabled Student Fee structure(Overseas)', ' Exam Superintendent Registration ', ' FACULTY OF SCIENCE & TECHNOLOGY Programs', ' Faculty of Arts Programs', ' Faculty of Education Programs', ' Faculty of Management Programs', ' Grading Scheme', ' How to Apply Online', ' Life at VU', ' New Registration', ' ORIC ', ' PROFESSIONAL COURSES DEVELOPMENT', ' PROTECTION AGAINST HARASSMENT  ', ' Prospectus', ' RIGHT OF ACCESS TO INFORMATION ', ' Recent Advertisements of VU', ' Schedule of other Charges', ' Short Certificate Course', ' Student Startup', ' Study Scheme', ' Suitability of  VU', ' Tender', ' Vendor Registration ', ' Zero Semester', 'Academic Calendar', 'Acceptable a

In [13]:
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

**Create our training data**

In [16]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)

# Padding to ensure consistent shape for bag-of-words representations
max_len = max(len(sample[0]) for sample in training)  # Find the maximum length
padded_training = []
for sample in training:
    bag, output_row = sample
    # Check if padding is actually necessary
    if len(bag) < max_len:
        padded_bag = bag + [0] * (max_len - len(bag))  # Pad with zeros if needed
    else:
        padded_bag = bag  # No padding needed
    padded_training.append([padded_bag, output_row])

training = np.array(padded_training, dtype=object) # Convert the padded list to NumPy array, allow object dtype


# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4-Model Training**</p>

#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.1-Neural Netowrk**</p>


#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.1.1 Model Training**</p>

In [19]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True) # Changed 'lr' to 'learning_rate'
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chat_model.keras', hist) # Changed 'chat_model' to 'chat_model.keras' to include the .keras extension

print("model created")

Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0084 - loss: 4.5464
Epoch 2/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0177 - loss: 4.4837
Epoch 3/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0367 - loss: 4.4349
Epoch 4/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0459 - loss: 4.4188
Epoch 5/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0654 - loss: 4.3235
Epoch 6/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0846 - loss: 4.1590
Epoch 7/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0768 - loss: 4.1033
Epoch 8/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1181 - loss: 3.9969
Epoch 9/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1465 - loss: 3.8869
Epoch 10/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1744 - loss: 3.7191
Epoch 11/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1778 - loss: 3.4454
Epoch 12/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2192 - lo

#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.2- LSTM**</p>

In [20]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Flatten,Dropout, BatchNormalization
from tensorflow.keras.layers import Bidirectional

#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.2.1 - Model Architecture 1**</p>


In [21]:
#Create the LSTM network
model = Sequential()
model.add(LSTM(64, input_shape=(len(train_x[0]),1), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(16,return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))
model.summary()
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
#adam = Adam(learning_rate=lr_schedule)
sgd = SGD(learning_rate=0.01 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=128, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 290, 64)             │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 290, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 290, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 290, 32)             │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 290, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 290, 32)             │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 16)                  │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 16)                  │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 93)                  │           1,581 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,477 (134.68 KB)

 Trainable params: 34,253 (133.80 KB)

 Non-trainable params: 224 (896.00 B)

NameError: name 'ExponentialDecay' is not defined

#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.2.2 - Model Architecture 2**</p>

In [ ]:
#Create the LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(len(train_x[0]),1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(16,return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))
model.summary()
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
#adam = Adam(learning_rate=lr_schedule)
sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 353, 128)          66560     
                                                                 
 dropout_7 (Dropout)         (None, 353, 128)          0         
                                                                 
 lstm_8 (LSTM)               (None, 353, 64)           49408     
                                                                 
 dropout_8 (Dropout)         (None, 353, 64)           0         
                                                                 
 batch_normalization_6 (Bat  (None, 353, 64)           256       
 chNormalization)                                                
                                                                 
 lstm_9 (LSTM)               (None, 353, 32)           12416     
                                                      

#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.2.2.1 - Model Saving**</p>

In [ ]:
model.save( path_to_save_model+'chat_model_2', hist)
print("model created and saved")

model created and saved


#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.2.3 - Model Architecture 3**</p>


In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(units=64,input_shape=(len(train_x[0]),1),return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=32,return_sequences=True)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=16,return_sequences=False)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))

# Build the model
model.build(input_shape=(None, len(train_x[0]), 1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 391, 128)          33792     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 391, 128)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 391, 64)           41216     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 391, 64)           0         
                                                                 
 batch_normalization (Batch  (None, 391, 64)           256       
 Normalization)                                                  
                                                        

In [ ]:
sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Epoch 1/100
932/932 [==============================] - 89s 80ms/step - loss: 4.5237 - accuracy: 0.0520
Epoch 2/100
932/932 [==============================] - 73s 79ms/step - loss: 3.7492 - accuracy: 0.1474
Epoch 3/100
932/932 [==============================] - 74s 79ms/step - loss: 3.2352 - accuracy: 0.2316
Epoch 4/100
932/932 [==============================] - 73s 79ms/step - loss: 2.7851 - accuracy: 0.3036
Epoch 5/100
932/932 [==============================] - 73s 79ms/step - loss: 2.4090 - accuracy: 0.3725
Epoch 6/100
932/932 [==============================] - 73s 78ms/step - loss: 2.0772 - accuracy: 0.4478
Epoch 7/100
932/932 [==============================] - 74s 79ms/step - loss: 1.7939 - accuracy: 0.5148
Epoch 8/100
932/932 [==============================] - 74s 80ms/step - loss: 1.5713 - accuracy: 0.5678
Epoch 9/100
932/932 [==============================] - 74s 80ms/step - loss: 1.3666 - accuracy: 0.6160
Epoch 10/100
932/932 [==============================] - 74s 80ms/step - l

In [ ]:
model.save( path_to_save_model+'chat_model_3', hist)
print("model created and saved")

model created and saved


#<p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">**4.2.4 - Model Architecture 4**</p>


In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(units=128,input_shape=(len(train_x[0]),1),return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64,return_sequences=True)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=32,return_sequences=True)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=16,return_sequences=False)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))

# Build the model
model.build(input_shape=(None, len(train_x[0]), 1))

model.summary()

sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 391, 256)          133120    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 391, 256)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 391, 128)          164352    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 391, 128)          0         
                                                                 
 batch_normalization (Batch  (None, 391, 128)          512       
 Normalization)                                                  
                                                        

In [ ]:
model.save( path_to_save_model+'chat_model_4', hist)
print("model created and saved")

model created and saved


In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(units=128,input_shape=(len(train_x[0]),1),return_sequences=True)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=64,return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=32,return_sequences=True)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=16,return_sequences=False)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))

# Build the model
model.build(input_shape=(None, len(train_x[0]), 1))

model.summary()

sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 391, 256)          133120    
 al)                                                             
                                                                 
 batch_normalization (Batch  (None, 391, 256)          1024      
 Normalization)                                                  
                                                                 
 bidirectional_1 (Bidirecti  (None, 391, 128)          164352    
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, 391, 128)          0         
                                                                 
 bidirectional_2 (Bidirecti  (None, 391, 64)           41216     
 onal)                                                  

In [ ]:
model.save( path_to_save_model+'chat_model_5', hist)
print("model created and saved")